In [1]:
import os
import json
import logging
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from utils import (
    preprocess_data, 
    SequenceDataset, 
    score_model, 
    predict,
    get_predictions,
    plot_predictions)

In [2]:
filename = "../data/mta_subway_221231_100wk_dbscan.parquet"
df = pd.read_parquet(filename)
df = df.fillna(0)
df.shape

(16800, 704)

In [3]:
df_train, df_test, features = preprocess_data(df)
df_test.shape

(3360, 704)

In [4]:
train_dataset = SequenceDataset(
        df_train,
        #target=None,
        features=features,
        sequence_length=336,
        forecast_lead=1
        )
test_dataset = SequenceDataset(
        df_test,
        #target=None,
        features=features,
        sequence_length=336,
        forecast_lead=1
        )

In [5]:
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [ ]:
class Encoder(nn.Module):
    def __init__(self, num_features, hidden_units, num_layers=1,dropout=0):
        super(Encoder, self).__init__()
        self.num_features = num_features
        self.hidden_units = hidden_units
        self.num_layers = num_layers
        self.dropout = dropout
        self.lstm = nn.LSTM(
            input_size=num_features,
            hidden_size=hidden_units,
            batch_first=True,
            num_layers=self.num_layers,
            dropout=self.dropout
        )
            
    def forward(self,x):
        batch_size = x.shape[0]
        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_units).requires_grad_()
        c0 = torch.zeros(self.num_layers, batch_size, self.hidden_units).requires_grad_()
        x, (h, c) = self.lstm(x, (h0, c0))
        return x, h, c

class Decoder(nn.Module):
    def __init__(self, seq_len, input_dim, num_features, dropout=0):
        super(Decoder, self).__init__()
        self.seq_len = seq_len
        self.input_dim = input_dim
        self.hidden_dim = input_dim
        self.num_features = num_features
        self.dropout=dropout
        self.lstm = nn.LSTM(
            input_size=1, 
            hidden_size=input_dim,
            num_layers=3,
            batch_first=True,
            dropout=self.dropout
        )
        self.linear = nn.Linear(self.hidden_dim, self.num_features)

    def forward(self, x, input_h, input_c):
        x = x.reshape((1,1,1))
        x, (hn, cn) = self.lstm(x, (input_h, input_c))
        x = self.linear(x)
        return x, hn, cn

class Seq2Seq(nn.Module):
    def __init__(self, num_features, horizon, encoder_hidden_units, encoder_num_layers=1, dropout=0):
        super().__init__()
        self.num_features = num_features
        self.hidden_units = hidden_units
        self.num_layers = num_layers
        self.dropout = dropout
        self.encoder = Encoder(
            num_features=self.num_features,
            hidden_units=self.hidden_units, 
            num_layers=self.num_layers, 
            dropout=self.dropout
            )
        self.decoder = Decoder(
            seq_len=,
            input_dim=,
            num_features=,
            
            dropout=self.dropout
            #input is output state of encoder
            #output length is equal to specified horizon length
            )
    
    def forward(self,x):
        batch_size = x.shape[0]
        h0 = torch.zeros(self.num_layers, batch_size, self.hidden_units).requires_grad_()
        c0 = torch.zeros(self.num_layers, batch_size, self.hidden_units).requires_grad_()
        o,(hn,_) = self.encoder(x, (h0, c0))
        
        out = self.decoder() #decoder takes output of encoder
        return out

In [ ]:
model = LSTMRegression(
        num_features=len(features), 
        hidden_units=num_hidden_units,
        num_layers=num_layers,
        dropout=dropout
        )#.to(device=device)

loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# rewrite train_model() to output whole sequence of length 'horizion'
#avg_loss = train_model(train_loader, model, loss_function, optimizer=optimizer)